In [1]:
def get_relaxation(data, roi_name, segmentation, anat, basestructs, labelsdf, te, r=1, unsupervised=False, savepath=""):
    heatmap_data=np.zeros_like(segmentation)
    
    if unsupervised:
        baseline_std=np.zeros((len(basestructs),))
        heatmaps=np.zeros((len(basestructs), 1, segmentation.shape[0], segmentation.shape[1]))
        for j,roi_baseline in enumerate(basestructs):
            roi_areas=labelsdf["Labels"][labelsdf["Anatomical Regions"].str.contains(roi_baseline)]
            roi=roi_baseline
            for i, roi in enumerate(roi_areas):
                heatmaps[j,i,:,:] = segment_relaxation(data, anat, anat, basestructs, labelsdf, roi,te, r=r)
            
    else:
        roi_areas=labelsdf["Labels"][labelsdf["Anatomical Regions"].str.contains(roi_name)]
        heatmaps=np.zeros((len(roi_areas),segmentation.shape[0], segmentation.shape[1]))

        for i, roi in enumerate(roi_areas):
            img_slice = np.unique(np.where(segmentation==roi)[-1])[0]
            heatmaps[i] = segment_relaxation(data[:,:,img_slice,:], 
                                             segmentation[:,:,img_slice],
                                             anat[:,:,img_slice], 
                                             basestructs, labelsdf, roi,te, r=r, savepath=savepath)
            heatmap_data[:,:,img_slice] = heatmap_data[:,:,img_slice] + heatmaps[i]

    
    
    return heatmaps, heatmap_data, img_slice

In [ ]:
def get_baseline_vals(seg, segmentationArr, data):
    """
    img_s: image slice to be analysed
    seg: segmentation id of baseline
    segmentationArr: segmentation np array
    data: mri raw np array
    
    returns;
    baseline: pixel values for baseline area nparray (len(vals), len(echos))
    """
#     segmentation=segmentationArr[:,:,img_s]
    if len(data.shape)==3:
        num_echos=data.shape[-1]
    else:
        num_echos=0
    
    if num_echos>0:
        for echo in range(num_echos):
    #         img=data[:,:,img_s,echo]
            img=data[:,:,echo]
            vals=img[segmentationArr==seg]

            if echo==0:
                baseline=np.zeros((len(vals),num_echos))
                baseline[:,0]=vals
            else:
                baseline[:,echo]=vals
    else:
        baseline=data[segmentationArr==seg]
            
    return baseline

In [1]:
def segment_relaxation(data, segmentation, anat, basestructs, labelsdf, roi, te, r=2, savepath=""):
    
    num_echos=data.shape[-1]
    heatmap=np.zeros_like(segmentation)

    for index in np.array(list(zip(*np.where(segmentation==roi)))):
        meanVals=np.zeros((num_echos,))
        stdVals=np.zeros((num_echos,))
        
        if r>1:
            mask=disk_roi(segmentation,index, r=r)
        else:
            mask=sq_roi(segmentation,index)
        
        base_seg,struct=argmax_roi_basestruct(mask, basestructs, labelsdf, anat)
        baseline=get_baseline_vals(base_seg, segmentation, data)
        if np.sum(mask*(anat==base_seg))>2:
            mask=mask*(anat==base_seg)

        _, meanVals, stdVals=get_echo_vals(data, mask)
        
        te0, te_spacing=te
        echos=np.arange(te0, np.ceil(te0 + (num_echos-1)*te_spacing), te_spacing)
        
        try:
            paramsBase, paramsData, _, _ =fit_relaxation(meanVals, stdVals, baseline, echos)
        
            relaxivity_base=1000/paramsBase[1]
            relaxivity_roi=1000/paramsData[1]
            diff=relaxivity_roi-relaxivity_base
        except:
            diff = 0.1
            
        if diff<0:
            diff=0
        heatmap[index[0], index[1]]= abs(diff)

        if savepath:
            if not os.path.exists(os.path.join(savepath, "Relaxation Curves")):
                os.mkdir(os.path.join(savepath, "Relaxation Curves"))
            
            Cbase,t2Base,Abase = paramsBase
            Celec,t2elec,Aelec = paramsData
            plt.figure()
            # plt.title()
            plt.errorbar(echos, meanVals, yerr=stdVals, fmt='o', color="blue")
            plt.plot(np.linspace(0,40,9), monoExp(np.linspace(0,40,9), Celec, t2elec, Aelec), color="blue", linewidth=3)
            plt.axvline(x=paramsData[1], color='blue', linestyle='--', linewidth=2)
            
            plt.errorbar(echos, np.mean(baseline,axis=0), yerr=np.std(baseline,axis=0), fmt='o', color="red")
            plt.plot(np.linspace(0,40,9), monoExp(np.linspace(0,40,9), Cbase, t2Base, Abase), color="red", linewidth=3)
            plt.axvline(x=paramsBase[1], color='red', linestyle='--', linewidth=2) 
            
            plt.ylabel("Signal a.u.")
            plt.xlabel("Time (ms)")
            plt.legend(["Electrode echo",  "Electrode Decay"])
            figname = f"index_{index}_contrast_{diff}-relaxation_curve.pdf"
            plt.savefig(os.path.join(savepath, "Relaxation Curves", figname), dpi = 1000)

            plt.figure()
            plt.imshow(data[:,:,0], cmap='gray', interpolation='nearest')
            plt.scatter(index[1], index[0], s=1, c="r", edgecolors='none')
            figname = f"index_{index}_contrast_{diff}-image.pdf"
            plt.savefig(os.path.join(savepath, "Relaxation Curves", figname), dpi = 1000)
            # plt.show()
            plt.close('all')
    
    return heatmap


In [ ]:
def fit_relaxation_new(meanVals, stdVals, baselineMean, baselineStd, echos, p0=(50, 30,0)):
    
    paramsBase, cvBase = scipy.optimize.curve_fit(monoExp, echos, baselineMean, p0, baselineStd, bounds=([-np.inf,-np.inf,0],[np.inf,np.inf,np.inf]))
    Cbase,t2Base,Abase = paramsBase
    p_sigma_base = np.sqrt(np.diag(cvBase))

    if Abase>baselineStd[-1]:
        Alower=Abase-baselineStd[-1]
    else:
        Alower=0
    Aupper=Abase+baselineStd[-1]
    
    paramsData, cvData = scipy.optimize.curve_fit(monoExp, echos, meanVals, paramsBase, stdVals, bounds=([Cbase-baselineStd[0], -np.inf, Alower], [Cbase+baselineStd[0],np.inf,Aupper]))
    Celec,t2elec,Aelec = paramsData
    p_sigma_data = np.sqrt(np.diag(cvData))
        
    return paramsBase, paramsData, p_sigma_base, p_sigma_data
    

In [2]:
def fit_relaxation(meanVals, stdVals, baseline, echos, p0=(50, 30,0)):
    
    paramsBase, cvBase = scipy.optimize.curve_fit(monoExp, echos, np.mean(baseline,axis=0), p0, np.std(baseline,axis=0), bounds=([-np.inf,-np.inf,0],[np.inf,np.inf,np.inf]))
    Cbase,t2Base,Abase = paramsBase
    p_sigma_base = np.sqrt(np.diag(cvBase))

    if Abase>np.std(baseline,axis=0)[-1]:
        Alower=Abase-np.std(baseline,axis=0)[-1]
    else:
        Alower=0
    Aupper=Abase+np.std(baseline,axis=0)[-1]
    
    paramsData, cvData = scipy.optimize.curve_fit(monoExp, echos, meanVals, paramsBase, stdVals, bounds=([Cbase-np.std(baseline,axis=0)[0], -np.inf, Alower], [Cbase+np.std(baseline,axis=0)[0],np.inf,Aupper]))
    Celec,t2elec,Aelec = paramsData
    p_sigma_data = np.sqrt(np.diag(cvData))
        
    return paramsBase, paramsData, p_sigma_base, p_sigma_data
    

In [6]:
def mrid_contrast_fixedROI(data, segmentation, anat, labelsdf, voxel_size, slice_thickness,
                           img_slice, basestructs, pattern_centers, mrid_dimensions, ionp_amount,
                           num_echos, savepath, roi_name=""):
    # px_size = 136    # Pixel size in micrometers
    voxx, voxy = voxel_size
    te0=4
    te_spacing=4.09
    mrid_lengths = mrid_dimensions[:,-1]
    print("Length of bars in MRID-barcode by design (um): " + str(mrid_lengths))

    dataSliced = data[:,:, img_slice, :]
    print("Sliced data shape: "+str(np.shape(dataSliced)))
    print("Sliced data image at echotime 0:")
    plt.figure()
    plt.imshow(dataSliced[:,:,0])
    
    contrast = np.zeros((len(pattern_centers),))
    roiAreas = np.zeros((len(pattern_centers),))
    
    for i in range(len(pattern_centers)):
        pattern_center = np.round(pattern_centers[i]).astype(int)
        print("MRID pattern center at: " + str(pattern_center))
        r = np.floor(mrid_lengths[i]/(voxx*1000)/2).astype(int)

        print("Fixed ROI radius for the " +str(i) +"th pattern: " +str(r))
        try:
            echos=np.arange(te0, np.ceil(te0 + (num_echos-1)*te_spacing), te_spacing)
            paramsBase, paramsData, p_sigma_base, p_sigma_data, roiArea = fit_relaxation_fixedROI(dataSliced, 
                                                                                              segmentation[:,:,img_slice], 
                                                                                              anat[:,:,img_slice], 
                                                                                              basestructs, 
                                                                                              pattern_center, labelsdf, r, 
                                                                                              echos)

            relaxivity_base=1000/paramsBase[1]
            relaxivity_roi=1000/paramsData[1]
            diff=relaxivity_roi-relaxivity_base
        except:
            diff = 0.1
        print("The relaxivity difference: "+str(diff))
        if diff<0:
            diff=0
            
        contrast[i]= abs(diff)
        # Now it is volume, change the name
        roiAreas[i]=roiArea
        # roiAreas[i]=roiArea*voxx*voxy*slice_thickness
    plt.show()
    print("Fixed ROI areas: " + str(roiAreas))
    print("Measured contrast intensities: " +str(contrast))
    densities = ionp_amount/roiAreas
    
    if roi_name:
        # savepath=os.path.join(sessionpath, "analysed", roi_name)
        if not os.path.exists(savepath):
            os.makedirs(savepath)
        fname = "ionp_densities_fixedROI.npy"
        np.save(os.path.join(savepath, fname), densities)

        fname = "contrast_intensities_fixedROI.npy"
        np.save(os.path.join(savepath, fname), contrast)
    
    return contrast, densities

In [7]:
def fit_relaxation_fixedROI(data, segmentation, anat, basestructs, pattern_center, labelsdf, r, echos):
    # Extracting the number of echo images from the data
    # num_echos = data.shape[-1]
    if r>1:
        mask = disk_roi(data[:,:,0], pattern_center, r)
    else:
        mask = sq_roi(data[:,:,0], pattern_center)
        
    plt.imshow(mask, cmap=cmap_gray, vmin=0.5)
    roiArea = np.sum(mask)
    
    base_seg,struct=argmax_roi_basestruct(mask, basestructs, labelsdf, anat)
    baseline=get_baseline_vals(base_seg, segmentation, data)
    
    _, meanVals, stdVals = get_echo_vals(data, mask)

    # Fit the relaxation
    paramsBase, paramsData, p_sigma_base, p_sigma_data = fit_relaxation(meanVals, stdVals, baseline, echos)
  
    return paramsBase, paramsData, p_sigma_base, p_sigma_data, roiArea

In [ ]:
def monoExp(x, C, t2, A):
    return C * np.exp(-x/t2) + A
def monoExp2(x,params):  
#     C=15.028
    C=params[0]
    t2=params[1]
    A=params[2]
#     A=1
#     C=C
#     A=0
    return C * np.exp(-x/t2) + A

# Legacy code

In [ ]:
import random

def shuffled_base_relaxation(data, base_seg, segmentation, num=200, r=1):
    
    diffs=np.zeros((num,))
    indeces=np.array(list(zip(*np.where(segmentation==base_seg))))
    idx=list(np.linspace(0, len(indeces)-1, len(indeces)).astype(int))
    
    #Randomly sample num-many indeces
    indeces[random.sample(idx, num)]

    baseline=get_baseline_vals(base_seg, segmentation, data)

    for j, i in enumerate(indeces[random.sample(idx, num)]):
        meanVals=np.zeros((num_echos,))
        stdVals=np.zeros((num_echos,))

        mask=disk_roi(segmentation,i, r=r)
        _, meanVals, stdVals=get_echo_vals(data, mask)

        paramsBase, paramsData, _, _ =fit_relaxation(meanVals, stdVals, baseline)

        relaxivity_base=1000/paramsBase[1]
        relaxivity_roi=1000/paramsData[1]
        diff=relaxivity_roi-relaxivity_base
        if diff<0:
            diff=0
        diffs[j]=diff
        
    return diffs

In [ ]:
def relaxation(data, elecSegment, baseline, masks, echos=np.linspace(5,40,8), savepath=""):
    pxIntMeans=np.empty((len(echos),))
    pxIntstd=np.empty((len(echos),))
    for echo in range(len(echos)-1):
        img=data[:,:,echo]
        pxIntMeans[echo]=np.mean(img[masks[elecSegment,-1,:,:]==1])
        pxIntstd[echo]=np.std(img[masks[elecSegment,-1,:,:]==1])
#         pxIntMeans[echo]=np.mean(img[masks[elecSegment,echo+1,:,:]==1])
#         pxIntstd[echo]=np.std(img[masks[elecSegment,echo+1,:,:]==1])
    
    img=data[:,:,-1]
    pxIntMeans[-1]=np.mean(img[masks[elecSegment,-1,:,:]==1])
    pxIntstd[-1]=np.std(img[masks[elecSegment,-1,:,:]==1])
    
    p0=(50, 30,0)
    paramsBase, cvBase = scipy.optimize.curve_fit(monoExp, echos, np.mean(baseline,axis=0), p0, np.std(baseline,axis=0), bounds=([-np.inf,-np.inf,0],[np.inf,np.inf,np.inf]))
    Cbase,t2Base,Abase = paramsBase
    
    if Abase>np.std(baseline,axis=0)[-1]:
        Alower=Abase-np.std(baseline,axis=0)[-1]
    else:
        Alower=0
    Aupper=Abase+np.std(baseline,axis=0)[-1]
    
    paramsData, cvData = scipy.optimize.curve_fit(monoExp, echos, pxIntMeans, paramsBase, pxIntstd, bounds=([Cbase-np.std(baseline,axis=0)[0], -np.inf, Alower], [Cbase+np.std(baseline,axis=0)[0],np.inf,Aupper]))
    Celec,t2elec,Aelec = paramsData
    
#     plt.figure()
#     plt.errorbar(echos, pxIntMeans, yerr=pxIntstd, fmt='o', color="blue")
#     plt.plot(np.linspace(0,40,9), monoExp(np.linspace(0,40,9), Celec, t2elec, Aelec), color="blue", linewidth=3)
    
#     plt.errorbar(echos, np.mean(baseline,axis=0), yerr=np.std(baseline,axis=0), fmt='o', color="red")
#     plt.plot(np.linspace(0,40,9), monoExp(np.linspace(0,40,9), Cbase, t2Base, Abase), color="red", linewidth=3)
    
    
#     plt.ylabel("Signal a.u.")
#     plt.xlabel("Time (ms)")
#     plt.legend(["Electrode echo",  "Electrode Decay"])
#     if savepath:
#         plt.savefig(savepath+"elec"+str(elecSegment)+"_relaxation.pdf")
#     plt.show()
    
    return paramsBase, paramsData


def relaxation_fixedROI_legacy(data, center,  elecSegment, baseline, r=3, echos=np.linspace(5,40,8), savepath="", baseName=""):

    disk=skimage.morphology.disk(r)
    x,y=center
    pxIntMeans=np.empty((len(echos),))
    pxIntstd=np.empty((len(echos),))
    
    for echo in range(len(echos)):
        img=data[:,:,echo]
        mask=np.zeros((256,256))
        mask[x-r:x+r+1, y-r:y+r+1]=disk
        
        pxIntMeans[echo]=np.mean(img[mask==1])
        pxIntstd[echo]=np.std(img[mask==1])
        
    p0=(50, 30,0)
    paramsBase, cvBase = scipy.optimize.curve_fit(monoExp, echos, np.mean(baseline,axis=0), p0, np.std(baseline,axis=0), bounds=([-np.inf,-np.inf,0],[np.inf,np.inf,np.inf]))
    Cbase,t2Base,Abase = paramsBase
    p_sigma_base = np.sqrt(np.diag(cvBase))

    
    if Abase>np.std(baseline,axis=0)[-1]:
        Alower=Abase-np.std(baseline,axis=0)[-1]
    else:
        Alower=0
    Aupper=Abase+np.std(baseline,axis=0)[-1]
    
    paramsData, cvData = scipy.optimize.curve_fit(monoExp, echos, pxIntMeans, paramsBase, pxIntstd, bounds=([Cbase-np.std(baseline,axis=0)[0], -np.inf, Alower], [Cbase+np.std(baseline,axis=0)[0],np.inf,Aupper]))
    Celec,t2elec,Aelec = paramsData
    p_sigma_data = np.sqrt(np.diag(cvData))
    
#     plt.figure()
#     plt.errorbar(echos, pxIntMeans, yerr=pxIntstd, fmt='o', color="blue")
#     plt.plot(np.linspace(0,40,9), monoExp(np.linspace(0,40,9), Celec, t2elec, Aelec), color="blue", linewidth=3)
    
#     plt.errorbar(echos, np.mean(baseline,axis=0), yerr=np.std(baseline,axis=0), fmt='o', color="red")
#     plt.plot(np.linspace(0,40,9), monoExp(np.linspace(0,40,9), Cbase, t2Base, Abase), color="red", linewidth=3)
#     plt.grid()
    
#     plt.ylabel("Signal a.u.")
#     plt.xlabel("Time (ms)")
#     plt.legend(["Electrode relaxation",  baseName+" relaxation"])
#     if savepath and baseName:
#         plt.savefig(savepath+"fixedROI-r:"+str(r)+"_elec"+str(elecSegment)+"_"+baseName+"_relaxation.pdf")
#     plt.show()
    
    return paramsBase, paramsData, p_sigma_base, p_sigma_data